# Исследование надежности заемщиков банка


## Описание проекта

Заказчик — кредитный отдел банка. Заказчика интересует, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откроем таблицу и изучим общую информацию о данных

**Задание 1. Импортируем библиотеку pandas. Считайем данные из csv-файла в датафрейм и сохраним в переменную `data`.**

In [2]:
import pandas as pd
data = pd.read_csv(r'E:\Analytics\data.csv')

**Задание 2. Выведем первые 20 строчек датафрейма `data` на экран.**

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

**Задание 8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**Задание 9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Задание 11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

**Задание 15. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [22]:
debt_in_time = data.groupby('children').agg({'debt': ['count','sum']})
debt_in_time['share_%'] = ((debt_in_time['debt']['sum']/debt_in_time['debt']['count'])*100).round(1)

In [23]:
#Для удобства чтения таблицы переименуем столбцы:
debt_in_time_share = debt_in_time.rename(columns={'count':'Общее кол-во кредитов', 'sum':'Кол-во задолженностей','share_%':'Процент задолженностей'}) 
debt_in_time_share

debt                       Процент задолженностей
         Общее кол-во кредитов Кол-во задолженностей                       
children                                                                   
0                        14107                  1063                    7.5
1                         4809                   444                    9.2
2                         2052                   194                    9.5
3                          330                    27                    8.2
4                           41                     4                    9.8
5                            9                     0                    0.0

**Вывод:** Как видно из полученной таблицы - утверждение "между количеством детей в семье и возвратом кредита в срок есть зависимость" - не верно. Количество детей при их наличии не влияет на возврат кредита в срок. Физические лица без детей возвращают кредит в срок чаще, чем физ.лица с детьми. С появлением детей и с увеличением их количества в семье - кредитов берут меньше.

**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [31]:
debt_family = data.groupby('family_status').agg({'debt': ['count','sum']})
debt_family['share_%'] = ((debt_family['debt']['sum']/debt_family['debt']['count'])*100).round(1)

In [32]:
#Для удобства чтения таблицы переименуем столбцы:
debt_family = debt_family.rename(columns={'count':'Общее кол-во кредитов', 'sum':'Кол-во задолженностей','share_%':'Процент задолженностей'}) 

In [33]:
#рассортируем таблицу по убыванию процента долгов:
debt_family = debt_family.sort_values(by='Процент задолженностей', ascending=False)
debt_family

debt                        \
                      Общее кол-во кредитов Кол-во задолженностей   
family_status                                                       
Не женат / не замужем                  2796                   273   
гражданский брак                       4146                   385   
женат / замужем                       12266                   927   
в разводе                              1189                    84   
вдовец / вдова                          951                    63   

                      Процент задолженностей  
                                              
family_status                                 
Не женат / не замужем                    9.8  
гражданский брак                         9.3  
женат / замужем                          7.6  
в разводе                                7.1  
вдовец / вдова                           6.6

**Проделаем тоже самое методом pivot_table():**

In [34]:
debt_family_table = data.pivot_table(index='family_status', values='debt',aggfunc={'debt':['sum','count']})
debt_family_table['share_%'] = ((debt_family_table['sum']/debt_family_table['count'])*100).round(1)

In [35]:
#рассортируем таблицу по убыванию процента долгов:
debt_family_table = debt_family_table.sort_values(by='share_%', ascending=False)
debt_family_table

,count,sum,share_%
family_status,,,
Не женат / не замужем,2796,273,9.8
гражданский брак,4146,385,9.3
женат / замужем,12266,927,7.6
в разводе,1189,84,7.1
вдовец / вдова,951,63,6.6


**Вывод:** Как видно из полученной таблицы - семейное положение незначительно влияет на возврат кредита в срок. Наибольший процент должников приходится на категорию людей "Не женат / не замужем". Заемщики, находящиеся в браке, возвращают кредит в срок на 22% чаще, чем заемщики категории "Не женат / не замужем".Разведенные заемщики возвращают кредит в срок на 28% чаще, чем неженатые/незамужние. Вдовец/вдова возвращают кредит в срок на 33% чаще, чем неженатые/незамужние заемщики.
Наибольшее количество кредитов берут заемщики, состоящие в браке. Меньше всего кредитов берут вдовцы и вдовы.

Чем меньше социальной ответсвенности - тем больше процент долгов. Люди, не обремененные социальными обязательствами, занимаются удовлетворением своих "хотелок" - машина поновее, сумочка из последней коллекции дизайнера, новый айфон и тд...  "Хотелок" много, денег меньше, штраф по кредиту - не смертельный.
    Люди, состоящие в браке, мыслят иначе. У них скорее всего есть ребенок(дети). Скорее всего у них есть цель так или иначе изменить жилищные условия, повысить качество жизни для себя и детей, образование, здравохранение... Все это повышает ответственность и мотивацию к финансовой грамотности. Покупки - как правило, обдуманные и запланированные, а значит люди рассчитывают свои финансовые возможности и, как следствие, процент по долгам снижается.
    Категория заемщиков - "в разводе" - точной статистики у меня нет, но полагаю, что эти люди уже имеют какое-то "имущество", часть проблем решена, часть проблем - связана с изменением социального статуса. Имеется понимание своих возможностей и понимание отношений с финансами. Отсюда меньше процент долгов.
    Касательно категории "вдовец/вдова" - как правило, это уже не молодые люди. Возможно - у них взрослые дети или внуки. Их финансовые притязания не высоки (поэтому и количество кредитов в этой категории наименьшее). Они рассчитывают на себя. С учетом того, что эти люди сформировались в период, когда кредитов не было - у них самое ответственное отношение к деньгам. Поэтому тут и самый низкий процент долгов. 

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [36]:
debt_income = data.pivot_table(index='total_income_category', values='debt',aggfunc={'debt':['sum','count']})
debt_income['share_%'] = ((debt_income['sum']/debt_income['count'])*100).round(1)

In [37]:
#рассортируем таблицу по убыванию процента долгов:
debt_income_sorted = debt_income.sort_values(by='share_%', ascending=False)
debt_income_sorted

,count,sum,share_%
total_income_category,,,
E,22,2,9.1
C,15938,1353,8.5
A,25,2,8.0
B,5014,354,7.1
D,349,21,6.0


**Вывод:** Чаще всего долг по кредиту имеют заемщики с доходом менее 30000 рублей (категория заемщиков 'E'). Реже всего долг по кредиту имеют Заемщики с уровнем дохода от 30001 до 50000 рублей (категория 'D'), а именно на 34% реже заемщиков категории 'Е'. Заемщики категории 'А' с доходом более 1000000 рублей находятся в середине выборки по долгам и возвращают кредит в срок на 12% чаще заемщиков с самым низким доходом.
Таким образом, можно сделать вывод, что чаще всего долг по кредитам имеют имеют заемщики с самым низким доходом. Далее нет линейной зависимости доходов и долга по кредиту.  

**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

In [29]:
debt_purpose = data.pivot_table(index='purpose_category', values='debt',aggfunc={'debt':['sum','count']})
debt_purpose['share_%'] = ((debt_purpose['sum']/debt_purpose['count'])*100).round(1)

In [30]:
#рассортируем по убыванию процента долгов:
debt_purpose_sorted = debt_purpose.sort_values(by='share_%', ascending=False)
debt_purpose_sorted

,count,sum,share_%
purpose_category,,,
операции с автомобилем,4281,400,9.3
получение образования,3989,369,9.3
проведение свадьбы,2324,183,7.9
операции с недвижимостью,10754,780,7.3


**Вывод:** Чаще всего долг по кредиту возникает у заемщиков, целью которых являются операции с автомобилем. Реже всего долгов при операциях с недвижимостью, а именно на 22%. 

 Как известно, автомобиль теряет 30% своей стоимости, как только он выезжает из автосалона. Поскольку автомобиль - это средство передвижения (надеюсь, в связи с известными событиями, так оно и останется, и машина не станет снова роскошью), его активно эксплуатируют. Отсюда - ремонт, запчасти, расходники, ТО, ДТП и тд и тп. Не выплатив один кредит, человек, оставшийся без авто, вынужден брать новый кредит на другое авто. Порочный круг замкнулся... 
    По образованию - хорошее образование стоит денег. Чаще всего кредит на образование берут заемщики с невысокой зарплатой (как раз с целью найти в дальнейшем высокооплачиваемую работу). Выполнить свои обязательства перед банком в таких условиях не просто.
    По свадьбам - когда "свежепостроенный корабль семьи" начинает свой путь в "океан жизни" с долгов - это печально. Радует, что для этой цели кредитов берут меньше всего. Но если есть традиция пригласить на свадьбу несколько соседних деревень - никуда не денешься. В будущем - это потенциальные должники.
    Категория "операции с недвижимостью" - как правильно это взвешенное решение. На эту категорию приходится самое большое число кредитов и наименьший процент долгов. Люди здраво подходят к оценке рисков, возможных последствий в виде потере этого самого жилья и стараются выполнять свои обязательства.

**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:* Причин появления пропусков в исходных данных несколько:
- люди могут не вносить или скрыть свои данные 
- ошибки и пропуски могут возникать при первичной обработке данных (объединение таблиц)
- ошибки и пропуски могут возникнуть по причине технического свойства - некорректная ссылка на данные или файл, неверный путь...

**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* При операциях с количественными переменными нужно заполнять пропуски медианным значением по следующей причине: часто количественные данные могут иметь экстремальные значения, сильно отличающиеся от остальных. Заполнив пропуски средним арифметическим значением мы можем сильно исказить данные, что повлечет не верный вывод в решении задачи. Такой проблемы не возникнет, если заполнить пропуски медианным значением. Искажение данных будет минимальным и недостаточным для верного вывода по гепотезе.

## Общий вывод.

Проведя исследование по платежеспособности заемщиков можно сдеать вывод о том, что самыми надежными заемщиками являются вдовы/вдовцы с пятью детьми, чей доход находится в пределах от 30001 до 50000 рублей и берущие кредит для операций с недвижимостью.
Самые ненадежные заемщики - это неженатые/незамужние заемщики с доходом менее 30000 рублей и берущие кредит на операции с автомобилем.
Касательно количества кредитов и их возврата в срок - можно сделать вывод, что самые надежные категории заемщиков это люди со средним и высоким доходом, семейные и с детьми. Они обладают бОльшей финансовой грамотностью и ответственностью, основная цель кредита - операции с недвижимостью.